# _Exploratory Data Analysis of the Population and Crop production data._

## Dataset Summary.

There are 2 datasets used in this project; Population dataset and Crops dataset. The population dataset used in this project was obtained from the **WORLD BANK GROUP** and it consists of the population of all countries and economies ranging from the years 1960 to 2019. This dataset was obtained from [this link](https://data.worldbank.org/indicator/SP.POP.TOTL), specifically [this file](http://api.worldbank.org/v2/en/indicator/SP.POP.TOTL?downloadformat=csv). The world bank group makes this data publicly available according to the open data standards and licenses datasets under the Creative Commons Attribution 4.0 International license (CC-BY 4.0). 


Furthermore, the crops dataset used in this project was acquired from the **Food and Agriculture Organization of the United Nations (FAO)**

### Exploratory Data Analysis checklist:

- Formulate the question
- Read in the data
- Check the packaging
- Look at the top and the bottom of your data
- Check your “n”s
- Validate with at least one external data source
- Make a plot
- Try the easy solution first
- Follow up

Epicycle of Analysis => 1. State the question, 2. EDA, 3. Build Model, 4. Interpret, 5. Communicate. 

#### Formulate the Question: How has production in paddy and wheat in Asia varied over time with change in population ?

## Load Required Packages

In [1]:
import pandas as pd
import altair as alt

## Read in the data and Check the packaging

In [2]:
world_pop_untidy = pd.read_csv("../data/raw/population_dataset.csv", skiprows = 4)

world_pop_untidy

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,Unnamed: 65
0,Aruba,ABW,"Population, total",SP.POP.TOTL,54211.0,55438.0,56225.0,56695.0,57032.0,57360.0,...,102560.0,103159.0,103774.0,104341.0,104872.0,105366.0,105845.0,106314.0,NaN,NaN
1,Afghanistan,AFG,"Population, total",SP.POP.TOTL,8996973.0,9169410.0,9351441.0,9543205.0,9744781.0,9956320.0,...,31161376.0,32269589.0,33370794.0,34413603.0,35383128.0,36296400.0,37172386.0,38041754.0,NaN,NaN
2,Angola,AGO,"Population, total",SP.POP.TOTL,5454933.0,5531472.0,5608539.0,5679458.0,5735044.0,5770570.0,...,25107931.0,26015780.0,26941779.0,27884381.0,28842484.0,29816748.0,30809762.0,31825295.0,NaN,NaN
3,Albania,ALB,"Population, total",SP.POP.TOTL,1608800.0,1659800.0,1711319.0,1762621.0,1814135.0,1864791.0,...,2900401.0,2895092.0,2889104.0,2880703.0,2876101.0,2873457.0,2866376.0,2854191.0,NaN,NaN
4,Andorra,AND,"Population, total",SP.POP.TOTL,13411.0,14375.0,15370.0,16412.0,17469.0,18549.0,...,82427.0,80774.0,79213.0,78011.0,77297.0,77001.0,77006.0,77142.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259,Kosovo,XKX,"Population, total",SP.POP.TOTL,947000.0,966000.0,994000.0,1022000.0,1050000.0,1078000.0,...,1807106.0,1818117.0,1812771.0,1788196.0,1777557.0,1791003.0,1797085.0,1794248.0,NaN,NaN
260,"Yemen, Rep.",YEM,"Population, total",SP.POP.TOTL,5315355.0,5393036.0,5473671.0,5556766.0,5641597.0,5727751.0,...,24473178.0,25147109.0,25823485.0,26497889.0,27168210.0,27834821.0,28498687.0,29161922.0,NaN,NaN
261,South Africa,ZAF,"Population, total",SP.POP.TOTL,17099840.0,17524533.0,17965725.0,18423161.0,18896307.0,19384841.0,...,52834005.0,53689236.0,54545991.0,55386367.0,56203654.0,57000451.0,57779622.0,58558270.0,NaN,NaN
262,Zambia,ZMB,"Population, total",SP.POP.TOTL,3070776.0,3164329.0,3260650.0,3360104.0,3463213.0,3570464.0,...,14465121.0,14926504.0,15399753.0,15879361.0,16363507.0,16853688.0,17351822.0,17861030.0,NaN,NaN


## Wrangle the World population dataset and look at top and bottom of dataset.

In [3]:
## Code Atrributed to Saule 

world_pop_tidy = world_pop_untidy.drop(columns = ['Country Code', 'Indicator Name', 
                                                  'Indicator Code', '2020', 'Unnamed: 65']).melt(id_vars = ['Country Name'],
                                                                                              var_name = 'Year',
                                                                                              value_name = 'Population')

world_pop_tidy = world_pop_tidy.rename(columns={'Country Name': 'Country'})

world_pop_tidy["Country"] = world_pop_tidy["Country"].replace("West Bank and Gaza", 'Palestine')

world_pop_tidy

,Country,Year,Population
0,Aruba,1960,54211.0
1,Afghanistan,1960,8996973.0
2,Angola,1960,5454933.0
3,Albania,1960,1608800.0
4,Andorra,1960,13411.0
...,...,...,...
15835,Kosovo,2019,1794248.0
15836,"Yemen, Rep.",2019,29161922.0
15837,South Africa,2019,58558270.0
15838,Zambia,2019,17861030.0


In [4]:
crop_untidy = pd.read_csv("../data/raw/crop_dataset.csv")

crop_untidy

,Area Code,Area,Item Code,Item,Element Code,Element,Unit,Y1961,Y1962,Y1963,...,Y2009,Y2010,Y2011,Y2012,Y2013,Y2014,Y2015,Y2016,Y2017,Y2018
0,2,Afghanistan,221,"Almonds, with shell",5312,Area harvested,ha,NaN,NaN,NaN,...,11029.0,11210.0,13469.0,13490.0,14114.0,13703.0,14676.0,19481.0,19793.0,20053.0
1,2,Afghanistan,221,"Almonds, with shell",5419,Yield,hg/ha,NaN,NaN,NaN,...,39154.0,49955.0,45000.0,45960.0,29910.0,19996.0,16521.0,16859.0,13788.0,17161.0
2,2,Afghanistan,221,"Almonds, with shell",5510,Production,tonnes,NaN,NaN,NaN,...,43183.0,56000.0,60611.0,62000.0,42215.0,27400.0,24246.0,32843.0,27291.0,34413.0
3,2,Afghanistan,711,"Anise, badian, fennel, coriander",5312,Area harvested,ha,NaN,NaN,NaN,...,17748.0,17000.0,19500.0,18500.0,18500.0,30000.0,25000.0,25638.0,27582.0,25785.0
4,2,Afghanistan,711,"Anise, badian, fennel, coriander",5419,Yield,hg/ha,NaN,NaN,NaN,...,6203.0,6000.0,6414.0,6757.0,6757.0,7167.0,7200.0,7037.0,6954.0,7036.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9624,249,Yemen,1729,"Treenuts, Total",5419,Yield,hg/ha,NaN,NaN,NaN,...,6314.0,6250.0,6304.0,6383.0,6992.0,6787.0,6578.0,6652.0,6665.0,6677.0
9625,249,Yemen,1729,"Treenuts, Total",5510,Production,tonnes,NaN,NaN,NaN,...,155.0,150.0,145.0,150.0,165.0,161.0,161.0,164.0,166.0,168.0
9626,249,Yemen,1735,Vegetables Primary,5312,Area harvested,ha,7000.0,7100.0,7200.0,...,52451.0,54293.0,48419.0,54065.0,50973.0,48318.0,43136.0,40551.0,39411.0,41201.0
9627,249,Yemen,1735,Vegetables Primary,5419,Yield,hg/ha,107143.0,107042.0,113889.0,...,117710.0,119064.0,114645.0,117101.0,106408.0,106925.0,112743.0,106053.0,106697.0,106845.0


In [6]:
# Code Attributed to Saule

paddy_wheat = crop_untidy.loc[(crop_untidy['Item Code'].isin(['15', '27', '30'])) & (crop_untidy['Element Code'] == 5510)].drop(
                               columns=['Element Code', 'Element', 'Unit', 'Item Code', 'Area Code'])

paddy_wheat_melt = paddy_wheat.melt(id_vars = ['Area', 'Item'],
                                    var_name = 'Year',
                                    value_name = 'Tonnes of crop').dropna()

paddy_wheat_melt['Year'] = paddy_wheat_melt['Year'].str.replace('Y', '')


replace_countries = {'Viet Nam': 'Vietnam',
                     'Yemen': 'Yemen, Rep.',
                     'China, Hong Kong SAR': 'China',
                     'China, Taiwan Province of': 'China',
                     'China, mainland': 'China',
                     'Republic of Korea': 'Korea, Rep.',
                     'Democratic People\'s Republic of Korea': 'Korea, Dem. People’s Rep.',
                     'Iran (Islamic Republic of)': 'Iran, Islamic Rep.',
                     'Kyrgyzstan': 'Kyrgyz Republic',
                     'Lao People\'s Democratic Republic': 'Lao PDR'}

paddy_wheat_melt = paddy_wheat_melt.replace({"Area": replace_countries})

paddy_wheat_melt

paddy_wheat_table = paddy_wheat_melt.pivot_table(values=['Tonnes of crop'], 
                                                 index=['Area', 'Year'], columns=['Item']).reset_index()


paddy_wheat_table.columns = ['Country', 'Year', 'Paddy_1', 'Paddy_2', 'Wheat']

paddy_wheat_table['Paddy'] = paddy_wheat_table['Paddy_1'] + paddy_wheat_table['Paddy_2']

paddy_wheat_table = paddy_wheat_table.drop(columns = ['Paddy_1', 'Paddy_2'])

paddy_wheat_table

,Country,Year,Wheat,Paddy
0,Afghanistan,1961,2279000.0,531773.0
1,Afghanistan,1962,2279000.0,531773.0
2,Afghanistan,1963,1947000.0,531773.0
3,Afghanistan,1964,2230000.0,633460.0
4,Afghanistan,1965,2282000.0,633460.0
...,...,...,...,...
2264,"Yemen, Rep.",2014,192215.0,NaN
2265,"Yemen, Rep.",2015,124940.0,NaN
2266,"Yemen, Rep.",2016,95917.0,NaN
2267,"Yemen, Rep.",2017,95651.0,NaN


In [10]:
tidy_df = pd.merge(paddy_wheat_table, world_pop_tidy, how = "left", on = ["Country", "Year"])

tidy_df

,Country,Year,Wheat,Paddy,Population
0,Afghanistan,1961,2279000.0,531773.0,9169410.0
1,Afghanistan,1962,2279000.0,531773.0,9351441.0
2,Afghanistan,1963,1947000.0,531773.0,9543205.0
3,Afghanistan,1964,2230000.0,633460.0,9744781.0
4,Afghanistan,1965,2282000.0,633460.0,9956320.0
...,...,...,...,...,...
2264,"Yemen, Rep.",2014,192215.0,NaN,25823485.0
2265,"Yemen, Rep.",2015,124940.0,NaN,26497889.0
2266,"Yemen, Rep.",2016,95917.0,NaN,27168210.0
2267,"Yemen, Rep.",2017,95651.0,NaN,27834821.0


# _References_

Data.worldbank.org. 2020. Population, Total | Data. [online] Available at: <https://data.worldbank.org/indicator/SP.POP.TOTL> [Accessed 19 November 2020].

Fao.org. 2020. FAOSTAT. [online] Available at: <http://www.fao.org/faostat/en/#data/QC> [Accessed 19 November 2020].